In [1]:
import pandas as pd

# load files
school_csv = "Resources/schools_complete.csv"
student_csv = "Resources/students_complete.csv"

school_data = pd.read_csv(school_csv)
student_data = pd.read_csv(student_csv)

# combined df
data_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
# ** Calculate the total number of schools
# ** Calculate the total number of students
# ** Calculate the total budget
# ** Calculate the average math score 
# ** Calculate the average reading score

#print number of unique school names in data_df
school_name_unique = len(pd.unique(data_df['school_name']))
print(f"number of schools: {school_name_unique}")

#student names are not unique because there can be more than one student in the same school or in the larger sample
student_name_all = len(data_df['student_name'])
print(f"total number of students: {student_name_all}")

total_budget = round(pd.unique(data_df['budget']).sum(),2)
print(f"total budget for all schools: ${total_budget}")

avg_math = round(data_df['math_score'].mean(), 2)
print(f"average math score: {avg_math}")

avg_reading = round(data_df['reading_score'].mean(), 2)
print(f"average reading score: {avg_reading}")


number of schools: 15
total number of students: 39170
total budget for all schools: $24649428
average math score: 78.99
average reading score: 81.88


In [3]:
# * Calculate the percentage of students with a passing math score (70 or greater)
# * Calculate the percentage of students with a passing reading score (70 or greater)
# * Calculate the percentage of students who passed math **and** reading (% Overall Passing)

pass_math_dict = {}
pass_reading_dict = {}
#first index value of each key is math score, second is reading score
pass_both_dict = {}


for id, math, read in zip(data_df['Student ID'],data_df['reading_score'],data_df['math_score']):
    
    id = int(id)

    
    if id not in pass_math_dict.keys():
        if math >=70:
            pass_math_dict[id] = [math]      
    if id not in pass_reading_dict.keys():
        if read >= 70:
            pass_reading_dict[id] = [read]
    if id not in pass_both_dict.keys():
        if read >= 70 and math >= 70:
            pass_both_dict[id] = [math,read]
            
# print(pass_math_dict)
# print(len(pass_reading_dict))

# # first index value of each key is math score, second is reading score
# print(pass_both_dict)

percent_math_pass = round((len(pass_math_dict))/(student_name_all) * 100, 2)
percent_reading_pass = round((len(pass_reading_dict))/(student_name_all) * 100, 2)
percent_both_pass = round((len(pass_both_dict))/(student_name_all) * 100, 2)

print(f"{percent_math_pass} % of students passed math.")
print(f"{percent_reading_pass} % of students passed reading.")
print(f"{percent_both_pass} % of students passed both math and reading.")




85.81 % of students passed math.
74.98 % of students passed reading.
65.17 % of students passed both math and reading.


In [4]:
# Create a dataframe to hold the summary results


district_summary_list = {
    'total schools': school_name_unique,
    'total students': student_name_all,
    'total budget': total_budget,
    'average math score': avg_math,
    'average reading score': avg_reading,
    '% passing math': percent_math_pass,
    '% passing reading': percent_reading_pass,
    '% passing overall': percent_both_pass
}
#convert to dataframe and format
district_summary_df = pd.DataFrame(district_summary_list, index = [0])

district_summary_df["total students"] = district_summary_df["total students"].map("{:,}".format)
district_summary_df["total budget"] = district_summary_df["total budget"].map("${:,.2f}".format)

district_summary_df.head()


,total schools,total students,total budget,average math score,average reading score,% passing math,% passing reading,% passing overall
0,15,"39,170","$24,649,428.00",78.99,81.88,85.81,74.98,65.17


## School Summary

In [5]:
# school name and type 
school_types = school_data.set_index(["school_name"])["type"]
# school_types


# total student count per school
students_per_school = data_df["school_name"].value_counts()
students_per_school

# Calculate the total school budget and per student budget
budget_per_school = data_df.groupby(["school_name"]).median()["budget"]
budget_per_student = budget_per_school / students_per_school

In [6]:
# average test scores per school
math_per_school = round(data_df.groupby(["school_name"]).mean()["math_score"],2)
reading_per_school = round(data_df.groupby(["school_name"]).mean()["reading_score"],2)
reading_per_school

school_name
Bailey High School       81.03
Cabrera High School      83.98
Figueroa High School     81.16
Ford High School         80.75
Griffin High School      83.82
Hernandez High School    80.93
Holden High School       83.81
Huang High School        81.18
Johnson High School      80.97
Pena High School         84.04
Rodriguez High School    80.74
Shelton High School      83.73
Thomas High School       83.85
Wilson High School       83.99
Wright High School       83.96
Name: reading_score, dtype: float64

In [7]:
# students who passed math and passed reading by creating separate filtered DataFrames.
passing_math_school = data_df[(data_df["math_score"] >= 70)]
passing_reading_school = data_df[(data_df["reading_score"] >= 70)]
# print(passing_math_school)
# students who passed both reading and math
passing_math_and_reading = data_df[(data_df["reading_score"] >= 70) & (data_df["math_score"] >= 70)]
passing_math_and_reading

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635
8,8,Michael Roth,M,10th,Huang High School,95,87,0,District,2917,1910635
9,9,Matthew Greene,M,10th,Huang High School,96,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [8]:
# percent passing rates

percent_pass_math_school = round(passing_math_school.groupby(["school_name"]).count()["student_name"] / students_per_school * 100, 2)
percent_pass_reading_school = round(passing_reading_school.groupby(["school_name"]).count()["student_name"] / students_per_school * 100, 2)
overall_passing = round(passing_math_and_reading.groupby(["school_name"]).count()["student_name"] / students_per_school * 100, 2)
# percent_pass_math_school
overall_passing

Bailey High School       54.64
Cabrera High School      91.33
Figueroa High School     53.20
Ford High School         54.29
Griffin High School      90.60
Hernandez High School    53.53
Holden High School       89.23
Huang High School        53.51
Johnson High School      53.54
Pena High School         90.54
Rodriguez High School    52.99
Shelton High School      89.89
Thomas High School       90.95
Wilson High School       90.58
Wright High School       90.33
dtype: float64

In [9]:
#  * School Name
#   * School Type
#   * Total Students
#   * Total School Budget
#   * Per Student Budget

#   * % Passing Math
#   * % Passing Reading
#   * % Overall Passing (The percentage of students that passed math **and** reading.)

school_summary_df = pd.DataFrame({
                                    "School Type": school_types,
                                    "Total Students": students_per_school,
                                    "Total School Budget": budget_per_school,
                                    "Per Student Budget": budget_per_student,
                                    "Average Math Score": math_per_school,
                                    "Average Reading Score": reading_per_school,
                                    "% Passing Math": percent_pass_math_school,
                                    "% Passing Reading": percent_pass_reading_school,
                                    "% Overall Passing": overall_passing
                                    
})

school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].map("${:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].map("${:,.2f}".format)


school_summary_df.head()


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60


## Top Performing Schools (By % Overall Passing)

In [10]:
# Sort and show top five schools
top_schools = school_summary_df.sort_values(["% Overall Passing"], ascending=False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54


## Bottom Performing Schools (By % Overall Passing)

In [11]:
top_schools = school_summary_df.sort_values(["% Overall Passing"], ascending=True)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54


## Math Scores by Grade

In [12]:
# create data series of scores by grade
ninth = data_df[(data_df["grade"] == "9th")]
tenth = data_df[(data_df["grade"] == "10th")]
eleventh = data_df[(data_df["grade"] == "11th")]
twelfth = data_df[(data_df["grade"] == "12th")]

# Group each series by school
ninth_scores = ninth.groupby(["school_name"]).mean()["math_score"]
tenth_scores = tenth.groupby(["school_name"]).mean()["math_score"]
eleventh_scores = eleventh.groupby(["school_name"]).mean()["math_score"]
twelfth_scores = twelfth.groupby(["school_name"]).mean()["math_score"]

# make summary df
math_by_grade_df = pd.DataFrame({"9th": round(ninth_scores,3), 
                                "10th": round(tenth_scores,3),
                                "11th": round(eleventh_scores,3), 
                                "12th": round(twelfth_scores,3)
})


math_by_grade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.084,76.997,77.516,76.492
Cabrera High School,83.095,83.155,82.766,83.277
Figueroa High School,76.403,76.540,76.884,77.151
Ford High School,77.361,77.672,76.918,76.180
Griffin High School,82.044,84.229,83.842,83.356
Hernandez High School,77.438,77.337,77.136,77.187
Holden High School,83.787,83.430,85.000,82.855
Huang High School,77.027,75.909,76.447,77.226
Johnson High School,77.188,76.691,77.492,76.863


## Reading Score by Grade 

In [13]:
# create data series of scores by grade
ninth = data_df[(data_df["grade"] == "9th")]
tenth = data_df[(data_df["grade"] == "10th")]
eleventh = data_df[(data_df["grade"] == "11th")]
twelfth_graders = data_df[(data_df["grade"] == "12th")]

# Group each series by school

ninth_scores = ninth.groupby(["school_name"]).mean()["reading_score"]
tenth_scores = tenth.groupby(["school_name"]).mean()["reading_score"]
eleventh_scores = eleventh.groupby(["school_name"]).mean()["reading_score"]
twelfth = twelfth_graders.groupby(["school_name"]).mean()["reading_score"]

# make summary df
reading_by_grade_df = pd.DataFrame({"9th": round(ninth_scores,3), 
                                "10th": round(tenth_scores,3),
                                "11th": round(eleventh_scores,3), 
                                "12th": round(twelfth,3)
})

reading_by_grade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303,80.907,80.946,80.912
Cabrera High School,83.676,84.253,83.788,84.288
Figueroa High School,81.199,81.409,80.640,81.385
Ford High School,80.633,81.263,80.404,80.662
Griffin High School,83.369,83.707,84.288,84.014
Hernandez High School,80.867,80.660,81.396,80.857
Holden High School,83.677,83.325,83.816,84.699
Huang High School,81.290,81.512,81.417,80.306
Johnson High School,81.261,80.773,80.616,81.228


## Scores by School Spending

In [14]:
#copy before binning- saves some time downstream
school_summary_df_copy = school_summary_df

In [15]:
# make bins
spending_bins = [0, 585, 630, 645, 680]
groups = ["<$585", "$585-630", "$630-645", "$645-680"]

school_summary_df["Spending Range (per student)"] = pd.cut(budget_per_student, spending_bins, labels=groups, right=False)
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Range (per student)
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64,$585-630
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33,<$585
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20,$630-645
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,54.29,$630-645
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60,$585-630
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53,$645-680
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,89.23,<$585
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51,$645-680
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54,$645-680
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54,$585-630


In [16]:
spending_math_scores = school_summary_df.groupby(["Spending Range (per student)"]).mean()["Average Math Score"]
spending_reading_scores = school_summary_df.groupby(["Spending Range (per student)"]).mean()["Average Reading Score"]
spending_pass_math = school_summary_df.groupby(["Spending Range (per student)"]).mean()["% Passing Math"]
spending_pass_reading = school_summary_df.groupby(["Spending Range (per student)"]).mean()["% Passing Reading"]
spending_pass_overall = school_summary_df.groupby(["Spending Range (per student)"]).mean()["% Overall Passing"]

In [17]:
spending_summary = pd.DataFrame({"Average Math Score" : spending_math_scores.round(2),
                                 "Average Reading Score": spending_reading_scores.round(2),
                                 "% Passing Math": spending_pass_math.round(2),
                                 "% Passing Reading": spending_pass_reading.round(2),
                                 "% Overall Passing": spending_pass_overall.round(2)
})

spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Range (per student),,,,,
<$585,83.45,83.94,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

In [18]:
size_bins = [0, 1000, 2000, 5000]
groups_size = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [19]:
school_summary_df_copy["School Size"] = pd.cut(school_summary_df_copy["Total Students"], size_bins, labels=groups_size, right=False)
school_summary_df_copy

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Range (per student),School Size
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64,$585-630,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33,<$585,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20,$630-645,Large (2000-5000)
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,54.29,$630-645,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60,$585-630,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53,$645-680,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,89.23,<$585,Small (<1000)
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51,$645-680,Large (2000-5000)
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54,$645-680,Large (2000-5000)
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54,$585-630,Small (<1000)


In [20]:
size_math_scores = school_summary_df_copy.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = school_summary_df_copy.groupby(["School Size"]).mean()["Average Reading Score"]
size_pass_math = school_summary_df_copy.groupby(["School Size"]).mean()["% Passing Math"]
size_pass_reading = school_summary_df_copy.groupby(["School Size"]).mean()["% Passing Reading"]
size_pass_overall = school_summary_df_copy.groupby(["School Size"]).mean()["% Overall Passing"]

In [21]:
# Assemble into DataFrame
size_summary = pd.DataFrame({"Average Math Score" : size_math_scores,
                             "Average Reading Score": size_reading_scores,
                             "% Passing Math": size_pass_math,
                             "% Passing Reading": size_pass_reading,
                             "% Overall Passing": size_pass_overall
})

size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.820,83.92500,93.55000,96.10000,89.885
Medium (1000-2000),83.374,83.86800,93.59800,96.79000,90.620
Large (2000-5000),77.745,81.34375,69.96375,82.76625,58.285


## Scores by School Type

In [22]:
type_math_scores = school_summary_df_copy.groupby(["School Type"]).mean()["Average Math Score"]
type_reading_scores = school_summary_df_copy.groupby(["School Type"]).mean()["Average Reading Score"]
type_pass_math = school_summary_df_copy.groupby(["School Type"]).mean()["% Passing Math"]
type_pass_reading = school_summary_df_copy.groupby(["School Type"]).mean()["% Passing Reading"]
type_pass_overall = school_summary_df_copy.groupby(["School Type"]).mean()["% Overall Passing"]

In [23]:
type_summary_df = pd.DataFrame({"Average Math Score" : type_math_scores,
                             "Average Reading Score": type_reading_scores,
                             "% Passing Math": type_pass_math,
                             "% Passing Reading": type_pass_reading,
                             "% Overall Passing": type_pass_overall
})

type_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.472500,83.897500,93.620000,96.586250,90.431250
District,76.955714,80.965714,66.548571,80.798571,53.671429
